In [1]:
import pandas as pd
import os
import json
import csv
import re, datetime
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, timedelta

from lifelines import KaplanMeierFitter

from global_functions import *

In [2]:
#newCase
def NewCase(dfx):
    dfN = dfx
    if dfN.empty==False:
        dfN['added']='NEW'
        dfN['removed']=''
        dfN=dfN.drop_duplicates(subset='id', keep="last")
        dfN['what']=dfN.apply(lambda x: 'status',axis=1)
        dfN['when']=dfN.apply(lambda x: pd.to_datetime(x['creation_time']),axis=1)
        dfN['who']=dfN.apply(lambda x: '',axis=1)
    dfN = dfN[['id','added',  'removed', 'what', 'when', 'who']]    
    return dfN


In [3]:
#return the activity
def return_activity_as(x):
    if x['what']=='status':
        return x['added']
    return''

In [4]:
#keep only changes in status
def keep_status(x):
    if x['what']=='status':
        return 1
    return 0

In [5]:
#Load data
#use os.sep to get the separator based on the operating system so as not to have issues running the script in other operating systems
df = pd.read_csv('.'+os.sep+'data'+os.sep+'bugs_info.zip',index_col=False,compression='zip')
df_history = pd.read_csv('.'+os.sep+'data'+os.sep+'bug_history.zip',index_col=False,compression='zip')
df_history= trans_to_datetime(df_history,['when'])
df= trans_to_datetime(df,['creation_time'])
print(len(df.id.unique()))
df.head(n=2)

138445


,id,release,Product,creation_time,version,resolution,is_assigned,is_resolved,is_fixed,first_assignment_date,...,last_resolved_date,first_fixed_date,last_fixed_date,time_assign_diff,time_resolve_diff,time_fix_diff,min_release,max_release,min_release_before_version_field,min_release_after_version_field
0,475361,4.5,Platform,2015-08-19 10:50:25,4.5,FIXED,0,1,1,NaN,...,2015-08-19 11:53:03,2015-08-19 11:53:03,2015-08-19 11:53:03,NaN,0.0,0.0,4.5,4.5,False,False
1,475365,4.6,Platform,2015-08-19 11:34:37,4.6,FIXED,0,1,1,NaN,...,2015-08-26 14:29:30,2015-08-26 14:29:30,2015-08-26 14:29:30,NaN,0.0,0.0,4.6,4.6,False,False


In [6]:
df=df[['id','creation_time']]

In [7]:
#create the new status
dfN=NewCase(df.copy())
result = df_history.append(dfN).reset_index(drop=True)

C:\Users\zeinab\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [8]:
result.loc[result['added'] == 'ASSIGNED ASSIGNED', 'added'] = 'ASSIGNED'
result.loc[result['removed'] == 'ASSIGNED ASSIGNED', 'removed'] = 'ASSIGNED'
result.loc[result['added'] == 'NEW NEW', 'added'] = 'NEW'
result.loc[result['removed'] == 'NEW NEW', 'removed'] = 'NEW'

In [9]:
result.to_csv('./data/process-mining/FINALwNEW.zip',index=False,compression='zip')

In [10]:
len(result.id.unique())

138445

# WITHOUT ASSIGNED_TO

In [11]:
result=pd.read_csv('./data/process-mining/FINALwNEW.zip',index_col=False,compression='zip')

C:\Users\zeinab\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
result.head()

,added,id,removed,what,when,who
0,platform-help-inbox@eclipse.org,100001,pde-ui-inbox@eclipse.org,assigned_to,2005-06-14 16:16:00,wassim.melhem@gmail.com
1,Help,100001,UI,component,2005-06-14 16:16:00,wassim.melhem@gmail.com
2,Platform,100001,PDE,product,2005-06-14 16:16:00,wassim.melhem@gmail.com
3,konradk@ca.ibm.com,100001,NaN,cc,2005-06-14 17:17:31,konradk@ca.ibm.com
4,dejan@ca.ibm.com,100001,platform-help-inbox@eclipse.org,assigned_to,2005-06-14 17:17:31,konradk@ca.ibm.com


In [13]:
result['keep']=result.apply(lambda x: keep_status(x),axis=1)

In [14]:
result=result[result['keep']==1]
result.drop(columns=['keep'])
result['activity']=result.apply(lambda x: return_activity_as(x),axis=1)

In [15]:
result=result[['id','activity','when','who']]

In [16]:
 result['when'] = pd.to_datetime(result['when'])

In [17]:
result.to_csv('./data/process-mining/disco_all.zip',index=False,compression='zip')

# rolling rleases

In [ ]:
roll=pd.read_csv('./data/bugs_info.zip',index_col=False,compression='zip')

In [ ]:
len(roll.id.unique())

In [ ]:
roll['release'] = roll.apply(lambda x : str(assign_to_closest_minor(x['version'])), axis=1)
roll['release'] = roll['release'].astype(str)

In [ ]:
release_rolling=['4.9','4.10']


In [ ]:
roll_id=roll[(roll['release']=='4.8')].id.unique()

In [ ]:
rolling=result.copy()
rolling=rolling[~rolling['id'].isin(roll_id)]
rolling.head()

In [ ]:
rolling.to_csv('./disco/disco_not_rolling_4-8.zip',index=False,compression='zip')

In [ ]:
#same process, no changes